In [6]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
import json

with open("documents/extraction.json", "r") as f:
    extraction = json.load(f)

len(extraction)

4702

In [94]:
%pip install --quiet llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-llms-openai llama-index-embeddings-openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_index.core.schema import Document

documents = [Document(text=list(extraction.values())[i]["transcript"]) for i in range(len(extraction))]


In [100]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

In [114]:
_documents = documents[2000:2001]


for doc in _documents:
    doc.text = llm.complete("Your task is to extract structured information from the following text. Extract all entities (Person, Companies, Organizations) and all their attributes (e.g. age, nationality, position, customer-number, identifiers etc.). Focus on simple key-value pairs. "+doc.text).text


In [96]:
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#
#embed_model = HuggingFaceEmbedding("T-Systems-onsite/cross-en-de-roberta-sentence-transformer")


from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(model="text-embedding-3-small")


In [27]:
%pip install --quiet python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [28]:
from dotenv import load_dotenv

load_dotenv()

True

In [118]:
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from llama_index.core import PropertyGraphIndex
from typing import Literal


import os


entities = Literal["PERSON", "COMPANY", "ORGANIZATION", "VALUE"]


kg_extractor = SchemaLLMPathExtractor(
  llm=llm, 
  possible_entities=entities, 
  #possible_relations=relations, 
  #kg_validation_schema=schema,
  strict=False,  # if false, allows values outside of spec
)

index = PropertyGraphIndex.from_documents(
    _documents,
    kg_extractors=[kg_extractor],
    embed_model=embed_model,
)

# insert additional documents / nodes
# index.insert(document)
# index.insert_nodes(nodes)

In [43]:
%pip install --quiet pyvis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [116]:
graph_dict = index.property_graph_store.to_dict()

graph_dict["triplets"]

{('Attachments',
  'DESCRIBES',
  'Information sheet on dealing with symptoms of illness (04.06.2021)'),
 ('Attachments',
  'DESCRIBES',
  'Overview of RHP Schule (valid from 07.06.2021)'),
 ('Attachments', 'DESCRIBES', 'RHP Schule (04.06.2021)'),
 ('Attachments',
  'DESCRIBES',
  'Word template  Corona-Selbsttest-Ausweis  (DIN-A4 and pocket format)'),
 ('Date of Document', 'DESCRIBES', '04.06.2021'),
 ('Frau Schulleiterin', 'DESCRIBES', 'School Principal (female)'),
 ('Herr Schulleiter', 'DESCRIBES', 'School Principal (male)'),
 ('Reference Number', 'DESCRIBES', 'ZS.4-BS4363.0/839'),
 ('Subject',
  'DESCRIBES',
  'Covid-19 protective measures in schools in Bavaria')}

In [ ]:
%pip --quiet install yfiles_jupyter_graphs

In [119]:
index.property_graph_store.show_jupyter_graph()

GraphWidget(layout=Layout(height='650px', width='100%'))

No such comm: c59346fd5a0e4f1884b973e82ced7b8c
